# Accessing Geostationary cloud satellite data

# GOES-16, GOES-17, and Himawari

- Funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program and AWS Public Dataset Program
  
### Credits: Tutorial development
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org) -  [Twitter](https://twitter.com/ChelleGentemann)   - Farallon Institute

### Satellite data

Satellite data come from either polar or low inclination orbits that circle the Earth or geostationary orbits that hold a location above a specific location on the Earth, near the Equator.  The type of data from satellites depends on both the orbit and the type of instrument.  There are definitions for the different satellite data types.
- L1 - satellite observations, usually each observation has a lat/lon/time associated with it
- L2 - derived geophysical retrievals (eg. SST), still with lat/lon/time
- L3 - variables mapped to a uniform space/time grid
- L4 - model output or analysis of data, may include multiple sensors

There are several different formats that satellite data:  [netCDF](https://www.unidata.ucar.edu/software/netcdf/), [HDF5](https://www.hdfgroup.org/solutions/hdf5/), and [geoTIFF](https://earthdata.nasa.gov/esdis/eso/standards-and-references/geotiff).
There are also som newer cloud optimized formats that data: [Zarr](https://zarr.readthedocs.io/en/stable/), Cloud Optimized GeoTIFF ([COG](https://www.cogeo.org/)).

### Understand the data
Data access can be easy and fast on the cloud, but to avoid potential issues down the road, spend time to understand the strengths and weaknesses of any data.  Any science results will need to address uncertainties in the data, so it is easier to spend time at the beginning to understand them and avoid data mis-application.

### Data proximate computing
These are BIG datasets that you can analyze on the cloud without downloading the data.  
You can run this on your phone, a Raspberry Pi, laptop, or desktop.   
By using public cloud data, your science is reproducible and easily shared!

### Here we will demonstrate some ways to access the different SST datasets on AWS:
- AWS GOES sea surface temperatures  (L2)
- AWS Himawari sea surface temperatures (L2)

### To run this notebook

Code is in the cells that have <span style="color: blue;">In [  ]:</span> to the left of the cell and have a colored background

To run the code:
- option 1) click anywhere in the cell, then hold `shift` down and press `Enter`
- option 2) click on the Run button at the top of the page in the dashboard

Remember:
- to insert a new cell below press `Esc` then `b`
- to delete a cell press `Esc` then `dd`

### First start by importing libraries


# How to find an AWS Public Dataset

- Click here: [AWS Public Dataset](https://aws.amazon.com/opendata/)


- Click on `Find public available data on AWS` button
- Search for GOES
- Select [GOES-16 and GOES-17](https://registry.opendata.aws/noaa-goes/) or [Himawari](https://registry.opendata.aws/noaa-himawari/)



## NETCDF GEO DATA

## Zarr made it seem so easy to read and access cloud SST data! 
## It is always that easy, right?

- When data is just dumped on the cloud, it is still useful, but can be challenging to use.  
Which brings us to GOES SSTs.....

- Info on the data is here -- GOES has a lot of different parameters and they are all stored in different files with names that are difficult to understand.  There are *80* different data products.  Himawari has 5 different products. This link lists them all and explains the different GOES Products [AWS info on GOES SST data](https://docs.opendata.aws/noaa-goes16/cics-readme.html).  

- The GOES and Himawari data is netCDF format.  There is a different for each of the 80 projects for year/day/hour.  A lot of files.  I find it really useful to 'browse' s3: buckets so that I can understand the directory and data structure.  [Explore S3 structure](https://noaa-goes16.s3.amazonaws.com/index.html).  The directory structure is `<Product>/<Year>/<Day of Year>/<Hour>/<Filename>`

- In the code below we are going to create a function that searches for all products availalbe from each satellite then reads in a full day of data. The files netCDF, so opening a day takes about 3-4 minutes.  There are other ways you could write this function depending on what your analysis goals are, this is just one way to get some data in a reasonable amount of time. 
- This function uses 
- [`s3fs.S3FileSystem`](https://s3fs.readthedocs.io/en/latest/) which holds a connection with a s3 bucket and allows you to list files, etc.  
- [`xr.open_mfdataset`](http://xarray.pydata.org/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset) opens a list of filenames and concatenates the data along the specified dimensions  


In [1]:
# filter some warning messages
import warnings 
warnings.filterwarnings("ignore") 

#libraries
import datetime as dt
import xarray as xr
import fsspec
import s3fs
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# make datasets display nicely
xr.set_options(display_style="html")  
import ipywidgets as widgets
from IPython.display import display, clear_output

#magic fncts #put static images of your plot embedded in the notebook
%matplotlib inline  
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

## Create functions for the drop down menus and reading the data

In [2]:
dir_base={'goes-east':'s3://noaa-goes16/','goes-west':'s3://noaa-goes17/','himawari':'s3://noaa-himawari8/'}
def prod_select(geo_sat):
    s3_base = dir_base[geo_sat]
    fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!
    file_location = fs.glob(s3_base+'*-L2-*')
    ipos=file_location[0].find('/')
    file_types = [file[ipos+1:] for file in file_location]
    vardict = file_types
    return vardict
def get_product(b):
    clear_output()
    geo_product = select_variable.value    
def get_sat(b):
    clear_output()
    geo_sat = select_satellite.value
    return 
def get_geo_filenames(sat,geo_product,lyr,idyjl):
    # arguments
    # sat   goes-east,goes-west,himawari
    # lyr   year
    # idyjl day of year
    
    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl)
    fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!

    #create strings for the year and julian day
    imon,idym=d.month,d.day
    syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    s3_base = dir_base[sat]
    #use glob to list all the files in the directory
    if sat=='goes-east':
        file_location = fs.glob(s3_base + geo_product + '/'+syr+'/'+sjdy+'/*/*.nc')
    if sat=='goes-west':
        file_location = fs.glob(s3_base + geo_product + '/'+syr+'/'+sjdy+'/*/*.nc')
    if sat=='himawari':
        file_location = fs.glob(s3_base + geo_product + '/'+syr+'/'+smon+'/'+sdym+'/*/*L2P*.nc')
    return file_location
def get_geo_data(file_location):
    # arguments
    # list of files to open
    # returns data and whether it exists
    
    ds,exist = [],False
    
    #make a list of links to the file keys
    fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!
    if len(file_location)<1:
        return 
    file_ob = [fs.open(file) for file in file_location]        #open connection to files
   
    #open all the day's data
    with xr.open_mfdataset(file_ob,combine='nested',concat_dim='time') as ds:      
        if 'himawari' not in files[0]:
            ds = ds.rename({'t':'time'})
            ds = ds.reset_coords()
        else:
            ds = ds.rename({'ni':'x','nj':'y'})  
    return ds


## This code creates a drop down menu to select a satellite 

In [3]:
vardict = ['goes-east','goes-west','himawari']
select_satellite = widgets.Dropdown(options=vardict,value=vardict[0],description='Satellite:',disabled=False,button_style='')
display(select_satellite)
select_satellite.observe(get_sat, names='value')

Dropdown(description='Satellite:', options=('goes-east', 'goes-west', 'himawari'), value='goes-east')

## This code lists the products for the selected satellite

In [11]:
proddict = prod_select(select_satellite.value)
select_product = widgets.Dropdown(options=proddict,value=proddict[0],description='Product ID:',disabled=False,button_style='')
display(select_product)
select_product.observe(get_product, names='value')


Dropdown(description='Product ID:', options=('AHI-L2-FLDK-Clouds', 'AHI-L2-FLDK-ISatSS', 'AHI-L2-FLDK-Rainfall…

In [7]:
lyr,idyjl=2020,215
files = get_geo_filenames(select_satellite.value,select_product.value,lyr,idyjl)
ds = get_geo_data(files)
ds

,Array,Chunk
Bytes,4.32 GB,15.00 MB
Shape,"(288, 1500, 2500)","(1, 1500, 2500)"
Count,1152 Tasks,288 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,4.32 GB,15.00 MB
Shape,"(288, 1500, 2500)","(1, 1500, 2500)"
Count,1152 Tasks,288 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
